In [1]:
!pip install tensorflow==2.7 keras gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.7 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.13.0rc0, 2.13.0rc1)
ERROR: No matching distribution found for tensorflow==2.7


In [2]:
!gdown 1T5Cm4Tt1sO21ARz6BvStSW4yks118rRz

Downloading...
From: https://drive.google.com/uc?id=1T5Cm4Tt1sO21ARz6BvStSW4yks118rRz
To: /content/small_data_set.zip
100% 2.07G/2.07G [00:23<00:00, 87.0MB/s]


In [3]:
import zipfile
import os
import shutil
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16

class ImageClassifier:
    def __init__(self, zip_file_path, dataset_folder="dataset", image_size=(150, 150), batch_size=32):
        self.zip_file_path = zip_file_path
        self.dataset_folder = dataset_folder
        self.image_size = image_size
        self.batch_size = batch_size

    def extract_zip(self):
        with zipfile.ZipFile(self.zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(self.dataset_folder)
        print("Dataset extracted successfully.")

    def prepare_data(self):
        data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

        self.train_data = data_gen.flow_from_directory(
            self.dataset_folder,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode="categorical",
            subset="training"
        )

        self.validation_data = data_gen.flow_from_directory(
            self.dataset_folder,
            target_size=self.image_size,
            batch_size=self.batch_size,
            class_mode="categorical",
            subset="validation"
        )
        self.num_classes = len(self.train_data.class_indices)
        print(f"Data prepared for training and validation. Number of classes: {self.num_classes}")

    def create_model(self):
        base_model = VGG16(weights="imagenet", include_top=False, input_shape=(*self.image_size, 3))
        base_model.trainable = False
        
        self.model = models.Sequential([
            base_model,
            layers.Flatten(),
            layers.Dense(512, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])

        self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        print("Deep neural network model created and compiled.")


    def train(self, epochs=10):
        self.model.fit(
            self.train_data,
            epochs=epochs,
            validation_data=self.validation_data
        )
        print("Model trained successfully.")


zip_file_path = "/content/small_data_set.zip"

classifier = ImageClassifier(zip_file_path)
classifier.extract_zip()
classifier.prepare_data()
classifier.create_model()
classifier.train()

Dataset extracted successfully.
Found 200 images belonging to 5 classes.
Found 50 images belonging to 5 classes.
Data prepared for training and validation. Number of classes: 5
58889256/58889256 [==============================] - 1s 0us/step
Deep neural network model created and compiled.
Epoch 1/10
7/7 [==============================] - 202s 29s/step - loss: 2.6389 - accuracy: 0.3050 - val_loss: 1.7527 - val_accuracy: 0.4000
Epoch 2/10
7/7 [==============================] - 181s 26s/step - loss: 1.0862 - accuracy: 0.5950 - val_loss: 1.1463 - val_accuracy: 0.5400
Epoch 3/10
7/7 [==============================] - 184s 26s/step - loss: 0.6245 - accuracy: 0.8000 - val_loss: 1.2213 - val_accuracy: 0.6000
Epoch 4/10
7/7 [==============================] - 182s 25s/step - loss: 0.3446 - accuracy: 0.8850 - val_loss: 1.0672 - val_accuracy: 0.6800
Epoch 5/10
7/7 [==============================] - 183s 25s/step - loss: 0.2680 - accuracy: 0.9350 - val_loss: 1.0151 - val_accuracy: 0.5600
Epoch 6/10